Crawl newspaper content using BeautifulSoup

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
def crawNewsData(baseUrl, url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    titles = soup.findAll('h3', class_='title-news')
    links = [link.find('a').attrs["href"] for link in titles]
    data = []
    for link in links:
        news = requests.get(baseUrl + link)
        soup = BeautifulSoup(news.content, "html.parser")
        try:
            title = soup.find("h1", class_="article-title").text
            abstract = soup.find("h2", class_="sapo").text
            time = soup.find("div",  class_="date-time").text
        except:
            title = ""
            abstract = ""
            time = ""
        
        body = soup.find("div", id="main-detail-body")
        content = ""
        try:
            for i in range(0,len(body.findChildren("p", recursive=False))):
                content += body.findChildren("p", recursive=False)[i].text
        except:
            content = ""
        data.append({
            "title": title,
            "abstract": abstract,
            "content": content,
            "time": time,
        })
        
    return data


In [ ]:
data = []
for day in range(1,27):
    url = "https://tuoitre.vn/trang-chu/xem-theo-ngay/%d-3-2021.htm"%(day)
    data = data + crawNewsData("https://tuoitre.vn", url)

In [ ]:
dataframe = pd.DataFrame(data)
dataframe = dataframe[dataframe.title != '']
dataframe

In [7]:
dataframe.to_csv('..\Crawl data\data.csv', index = False)